In [1]:
1+1

2

In [2]:
%pip install pandas plotly kaleido

Note: you may need to restart the kernel to use updated packages.


In [3]:
# League vizzes notebook cell (ORtg, TS%, 3P rate) from team_game_data.csv
# Produces standalone HTML files you can embed in Quasar:
#   public/viz/league/ortg.html
#   public/viz/league/ts.html
#   public/viz/league/three_rate.html
#   public/viz/league/league_trends_combo.html

from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio

# ---------- CONFIG ----------
from pathlib import Path

BASE_DIR = Path.cwd()   # safest inside notebooks
CSV_PATH ='C:\\Users\\huash\\OneDrive\\Documents\\Desktop\\DataVisNBAProject\\nba-viz-site\\data\\team_game_data.csv'
OUT_DIR = BASE_DIR / "nba-viz-site" / "public" / "viz" / "league"  # change to your Quasar project's path
AGG = "M"  # "M" monthly, "Q" quarterly, "Y" yearly
INCLUDE_PLOTLYJS = "cdn"  # "cdn" keeps files small
# ---------------------------

def safe_div(a, b):
    a = a.astype(float)
    b = b.astype(float)
    return np.where(b == 0, np.nan, a / b)

def add_metrics(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["gamedate"] = pd.to_datetime(df["gamedate"])

    # Offensive rating
    df["ortg"] = 100.0 * safe_div(df["points"], df["possessions"])

    # True Shooting %
    df["ts"] = safe_div(df["points"], 2.0 * (df["fgattempted"] + 0.44 * df["ftattempted"]))

    # 3P attempt rate
    df["threeRate"] = safe_div(df["fg3attempted"], (df["fg2attempted"] + df["fg3attempted"]))

    return df

def aggregate_league(df: pd.DataFrame, freq: str = "M") -> pd.DataFrame:
    # aggregate at period boundary timestamps (nice for x-axis)
    period = df["gamedate"].dt.to_period(freq)
    t = df.assign(period=period.dt.to_timestamp())

    league = (
        t.groupby("period", as_index=False)[["ortg", "ts", "threeRate"]]
         .mean()
         .sort_values("period")
         .rename(columns={"period": "date"})
    )
    return league

def write_html(fig, out_path: Path):
    out_path.parent.mkdir(parents=True, exist_ok=True)
    pio.write_html(
        fig,
        out_path,
        full_html=True,
        include_plotlyjs=INCLUDE_PLOTLYJS,
        config={"displayModeBar": True, "responsive": True},
    )
    print("Wrote:", out_path.resolve())

# ---------- LOAD + BUILD ----------
df = pd.read_csv(CSV_PATH)
df = add_metrics(df)
league = aggregate_league(df, freq=AGG)

league.head()


,date,ortg,ts,threeRate
0,2014-10-01,105.678425,0.536214,0.260982
1,2014-11-01,106.200462,0.539711,0.266302
2,2014-12-01,106.417771,0.539141,0.266144
3,2015-01-01,104.860246,0.531071,0.270411
4,2015-02-01,104.334062,0.527612,0.273134


In [4]:
%pip install nbformat>=4.2.0 

Note: you may need to restart the kernel to use updated packages.


In [5]:
# ORtg trend
fig_ortg = px.line(
    league,
    x="date",
    y="ortg",
    title=f"NBA Offensive Rating ({AGG}-aggregated)",
    labels={"date": "Date", "ortg": "Offensive Rating"},
)
fig_ortg.update_traces(mode="lines")

write_html(fig_ortg, OUT_DIR / "ortg.html")
fig_ortg


Wrote: C:\Users\huash\OneDrive\Documents\Desktop\DataVisNBAProject\nba-viz-site\src\pages\notebooks\nba-viz-site\public\viz\league\ortg.html


In [7]:
# TS% trend
fig_ts = px.line(
    league,
    x="date",
    y="ts",
    title=f"League True Shooting % ({AGG}-aggregated)",
    labels={"date": "Date", "ts": "True Shooting %"},
)
fig_ts.update_traces(mode="lines")

write_html(fig_ts, OUT_DIR / "ts.html")
fig_ts


Wrote: C:\Users\huash\OneDrive\Documents\Desktop\DataVisNBAProject\nba-viz-site\src\pages\notebooks\nba-viz-site\public\viz\league\ts.html


In [ ]:
# Combo chart (all three together)
league_long = league.melt(id_vars=["date"], value_vars=["ortg", "ts", "threeRate"],
                          var_name="metric", value_name="value")

pretty = {"ortg": "ORtg", "ts": "TS%", "threeRate": "3P Rate"}
league_long["metric"] = league_long["metric"].map(pretty)

fig_combo = px.line(
    league_long,
    x="date",
    y="value",
    color="metric",
    title=f"League Trends ({AGG}-aggregated): ORtg, TS%, 3P Rate",
    labels={"date": "Date", "value": "Value", "metric": ""},
)
fig_combo.update_traces(mode="lines")

write_html(fig_combo, OUT_DIR / "league_trends_combo.html")
fig_combo
